In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
task_names = [
#     "COLA", 
#     "SST2",
#     "MNLI",
    "RTE",
#     "WNLI",
#     "QQP",
#     "MRPC",
#     "STSB",
#     "QNLI",
#     "SPACY_POS",
#     "SPACY_NER",
#     "THIRD",
#     "BLEU",
]


In [3]:
SEED = 1

## Load previously trained model
Hint: make sure the `bert_model` is initialized correctly!

In [4]:
task_kwargs = {
    "dl_kwargs": {"batch_size": 8},
#     freeze_bert=True,
    "bert_model": 'bert-base-cased',
    "max_len": 200   
}

In [5]:
%%time
from metal.mmtl.glue.glue_tasks import create_glue_tasks_payloads

# Create tasks and payloads
tasks, payloads = create_glue_tasks_payloads(task_names, **task_kwargs)

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
Using random seed: 817521
Loading RTE Dataset



CPU times: user 29.9 s, sys: 4.55 s, total: 34.5 s
Wall time: 1min 11s


In [6]:
tasks, payloads

([ClassificationTask(name=RTE, loss_multiplier=1.0)],
 [Payload(RTE_train: labels_to_tasks=[{'RTE_gold': 'RTE'}], split=train),
  Payload(RTE_valid: labels_to_tasks=[{'RTE_gold': 'RTE'}], split=valid),
  Payload(RTE_test: labels_to_tasks=[{'RTE_gold': 'RTE'}], split=test)])

In [7]:
from metal.mmtl.metal_model import MetalModel

model = MetalModel(tasks, seed=SEED, verbose=False)

#### Sanity check that task head is trained!

In [8]:
# model.score(payloads[1])

## Define slices for evaluation

In [9]:
%%time
from metal.mmtl.glue.glue_tasks import create_glue_tasks_payloads

# define slices
slice_dict = {  # A map of the slices that apply to each task
    "RTE": ["dash_semicolon", "more_people"]
}

# Create tasks and payloads
task_kwargs.update({"slice_dict": slice_dict})
_, payloads_slice = create_glue_tasks_payloads(
    task_names, **task_kwargs
)

Using random seed: 128037
Loading RTE Dataset



Added label_set with 1003/2490 labels for task RTE:dash_semicolon to payload RTE_train.
Added label_set with 64/2490 labels for task RTE:more_people to payload RTE_train.
Added label_set with 116/277 labels for task RTE:dash_semicolon to payload RTE_valid.
Added label_set with 12/277 labels for task RTE:more_people to payload RTE_valid.
Added label_set with 1103/3000 labels for task RTE:dash_semicolon to payload RTE_test.
Added label_set with 67/3000 labels for task RTE:more_people to payload RTE_test.
CPU times: user 23 s, sys: 2.87 s, total: 25.9 s
Wall time: 48.1 s


In [10]:
payloads_slice

[Payload(RTE_train: labels_to_tasks=[{'RTE_gold': 'RTE', 'RTE_slice:dash_semicolon': 'RTE:dash_semicolon', 'RTE_slice:more_people': 'RTE:more_people'}], split=train),
 Payload(RTE_valid: labels_to_tasks=[{'RTE_gold': 'RTE', 'RTE_slice:dash_semicolon': 'RTE:dash_semicolon', 'RTE_slice:more_people': 'RTE:more_people'}], split=valid),
 Payload(RTE_test: labels_to_tasks=[{'RTE_gold': 'RTE', 'RTE_slice:dash_semicolon': 'RTE:dash_semicolon', 'RTE_slice:more_people': 'RTE:more_people'}], split=test)]

#### Sanity check the number of examples in the train set

In [11]:
import numpy as np
def count_num_labels(labels):
    return np.sum(np.array(labels) != 0)

In [12]:
dataset = payloads_slice[0].data_loader.dataset
for labelset_name, labels in dataset.labels.items():
    print(labelset_name, count_num_labels(labels))

RTE_gold 2490
RTE:dash_semicolon 1003
RTE:more_people 64


### Evaluate baseline model on the slice of interest

In [13]:
# model.score(payloads_slice[1])

In [14]:
assert False

AssertionError: 

## Finetune model on slice of interest

In [ ]:
from metal.mmtl.metal_model import MetalModel

from metal.mmtl.trainer import MultitaskTrainer
trainer = MultitaskTrainer(seed=SEED)

Uncomment below to only finetune on the slices, not the original task


In [ ]:
# # only finetune on the slices, not the original task
# for p in payloads_slice:
#     p.task_names.remove('COLA')
# payloads_slice

NOTE: We are training on a different set of payloads than we initialized the model with.

In [ ]:
trainer.train_model(
    model,
    payloads_slice,
    checkpoint_metric="RTE/RTE_valid/accuracy",
    checkpoint_metric_mode="max",
    checkoint_best=True,
    writer="tensorboard",
    optimizer="adamax",
    lr=5e-5,
    l2=1e-3,
    log_every=0.1, 
    score_every=0.1,
    n_epochs=3,
    progress_bar=True,
    checkpoint_tasks=True,
    checkpoint_cleanup=False
)

#### Did we improve?

TODO: The final metrics score printed above may be inaccurate... 

Better way to evaluate is by re-evaluating on the checkpoint with the best weights.

In [ ]:
from metal.mmtl.metal_model import MetalModel
model.score(payloads_slice[1])